In [1]:
import os

# Устанавливаем папки

In [2]:
# fish_name = input("Напишите с какой рыбой работаем:")
fish_name = 'карп'
if fish_name.lower() == 'карп':
    LABEL_MAP_NAME = 'label_map_karp.pbtxt'
    name = 'karp'
elif fish_name.lower() == 'осетр':
    LABEL_MAP_NAME = 'label_map_osetr.pbtxt'
    name = 'osetr'
else:
    print('Пока нет фотографий для такой рыбы')

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','data'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, name + '_checkpoints'), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['CHECKPOINT_PATH'], name + '_pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [21]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 97475, done.
remote: Counting objects: 100% (755/755), done.
remote: Compressing objects: 100% (352/352), done.
remote: Total 97475 (delta 462), reused 620 (delta 396), pack-reused 96720
Receiving objects: 100% (97475/97475), 613.70 MiB | 968.00 KiB/s, done.
Resolving deltas: 100% (70880/70880), done.


In [53]:
import re

def update_setup():
    file_path = 'setup.py'
    with open(file_path, 'r', encoding='utf-8') as file:
        setup_content = file.read()
    
    # Обновляем tf-models-official
    setup_content = setup_content.replace('tf-models-official>=2.5.1', 'tf-models-official>=2.5.1, <2.14.0')
    setup_content = setup_content.replace("'keras'", "#'keras'")

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(setup_content)

In [55]:
%ls

0.InstallingLibraries.ipynb  2.Training.ipynb  README.md         Tensorflow/
1.DataExplore.ipynb          3.Testing.ipynb   requirements.txt


In [58]:
if os.name=='posix':  
    # !sudo apt-get install protobuf-compiler
    %cd Tensorflow/models/research 
    !protoc object_detection/protos/*.proto --python_out=.
    %cp object_detection/packages/tf2/setup.py .
    update_setup()
    %run pip install .
    %cd ../../..

if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py
    update_setup()
    !cd Tensorflow/models/research && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

/home/midnightknight/Blood-object-detection/Tensorflow/models/research
1
Processing /home/midnightknight/Blood-object-detection/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Using cached tf_models_official-2.13.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached tensorflow_text-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached tensorflow-2.13.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached tf_models_official-2.13.2-py2.py

In [59]:
%pip freeze

absl-py==1.4.0
apache-beam==2.57.0
array_record==0.5.1
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
attrs==23.2.0
avro-python3==1.10.2
bleach==6.1.0
cachetools==5.3.3
certifi==2024.7.4
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==2.2.1
colorama==0.4.6
comm==0.2.2
contextlib2==21.6.0
contourpy==1.2.1
crcmod==1.7
cycler==0.12.1
Cython==3.0.10
debugpy==1.8.2
decorator==5.1.1
dill==0.3.1.1
dm-tree==0.1.8
dnspython==2.6.1
docopt==0.6.2
docstring_parser==0.16
etils==1.7.0
exceptiongroup==1.2.2
executing==2.0.1
fastavro==1.9.5
fasteners==0.19
flatbuffers==24.3.25
fonttools==4.53.1
fsspec==2024.6.1
gast==0.4.0
gin-config==0.5.0
google-api-core==2.19.1
google-api-python-client==2.137.0
google-auth==2.32.0
google-auth-httplib2==0.2.0
google-auth-oauthlib==1.0.0
google-pasta==0.2.0
googleapis-common-protos==1.63.2
grpcio==1.64.1
h5py==3.11.0
hdfs==2.7.3
httplib2==0.22.0
idna==3.7
immutabledict==4.2.0
importlib_resources==6.4.0
ipykernel==6.29.5
ipython==8.26.0
jedi==0.19.1
jobli

## Проверяем все ли библиотеки скачались

In [60]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
%run {VERIFICATION_SCRIPT}

2024-07-13 19:38:32.394293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 19:38:33.260775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-13 19:38:35.263295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
Running tests under Python 3.10.12: /home/midnightknight/Blood-object-detection/.obj_det/bin/python3
2024-07-13 19:38:35.451839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node r

WARNING: `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.


W0713 19:38:36.090312 132218468208640 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/object_detection/builders/model_builder.py:1112: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'


WARNING: Building experimental DeepMAC meta-arch. Some features may be omitted.


W0713 19:38:36.283026 132218468208640 model_builder.py:1112] Building experimental DeepMAC meta-arch. Some features may be omitted.


...

I0713 19:38:56.413019 132218468208640 test_util.py:2462] time(__main__.ModelBuilderTF2Test.test_unknown_faster_rcnn_feature_extractor): 0.0s
[       OK ] ModelBuilderTF2Test.test_unknown_faster_rcnn_feature_extractor
[ RUN      ] ModelBuilderTF2Test.test_unknown_meta_architecture


time(__main__.ModelBuilderTF2Test.test_unknown_meta_architecture): 0.0s

I0713 19:38:56.415050 132218468208640 test_util.py:2462] time(__main__.ModelBuilderTF2Test.test_unknown_meta_architecture): 0.0s
[       OK ] ModelBuilderTF2Test.test_unknown_meta_architecture
[ RUN      ] ModelBuilderTF2Test.test_unknown_ssd_feature_extractor


time(__main__.ModelBuilderTF2Test.test_unknown_ssd_feature_extractor): 0.0s

I0713 19:38:56.417317 132218468208640 test_util.py:2462] time(__main__.ModelBuilderTF2Test.test_unknown_ssd_feature_extractor): 0.0s
[       OK ] ModelBuilderTF2Test.test_unknown_ssd_feature_extractor
----------------------------------------------------------------------
Ran 24 tests in 20.963s

OK (skipped=1)


In [61]:
%pip install tensorflow[and-cuda]==2.13.0

  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.1
    Uninstalling tensorflow-2.13.1:
      Successfully uninstalled tensorflow-2.13.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%run {VERIFICATION_SCRIPT}

Running tests under Python 3.10.12: /home/midnightknight/Blood-object-detection/.obj_det/bin/python3
Warning, backtrace signal handler for signal 11 overwrote previous handler.
Warning, backtrace signal handler for signal 6 overwrote previous handler.
Warning, backtrace signal handler for signal 7 overwrote previous handler.
Warning, backtrace signal handler for signal 4 overwrote previous handler.
Warning, backtrace signal handler for signal 8 overwrote previous handler.
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
/home/midnightknight/Blood-object-detection/.obj_det/lib/python3.10/site-packages/object_detection/builders/model_builder.py:1112: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arch.'


WARNING: Building experimental DeepMAC meta-arch. Some features may be omitted.


W0713 19:40:07.584423 132218468208640 model_builder.py:1112] Building experimental DeepMAC meta-arch. Some features may be omitted.


time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.44s

I0713 19:40:07.818273 132218468208640 test_util.py:2462] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.44s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)


...

I0713 19:40:14.289223 132218468208640 efficientnet_model.py:143] round_filter input=80 output=88


round_filter input=80 output=88

I0713 19:40:14.563512 132218468208640 efficientnet_model.py:143] round_filter input=80 output=88


round_filter input=112 output=120

I0713 19:40:14.565011 132218468208640 efficientnet_model.py:143] round_filter input=112 output=120
